In [1]:
import pandas as pd

In [108]:
df = pd.read_csv('sample_data.txt',delimiter=' ')

In [109]:
df.head()

,オーダー№,測定投入日,測定投入時間,測定完了日,測定完了時間,測定機№,Unnamed: 6
0,10FRE103S0,20200628,742,20200628,1108,LH26,NaN
1,10G1E20200,20200628,19,20200628,1834,LH25,NaN
2,10FVE20400,20200628,646,20200628,1855,LH12,NaN
3,10FSE104S0,20200628,1119,20200628,2058,LH26,NaN
4,10FFFA0300,20200628,1048,20200629,427,LH02,NaN


In [110]:
# 不要な列を削除
#df.drop['Unnamed: 6']
del df['Unnamed: 6']
df.head()

,オーダー№,測定投入日,測定投入時間,測定完了日,測定完了時間,測定機№
0,10FRE103S0,20200628,742,20200628,1108,LH26
1,10G1E20200,20200628,19,20200628,1834,LH25
2,10FVE20400,20200628,646,20200628,1855,LH12
3,10FSE104S0,20200628,1119,20200628,2058,LH26
4,10FFFA0300,20200628,1048,20200629,427,LH02


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 6 columns):
オーダー№     56 non-null object
測定投入日     56 non-null int64
測定投入時間    56 non-null int64
測定完了日     56 non-null int64
測定完了時間    56 non-null int64
測定機№      56 non-null object
dtypes: int64(4), object(2)
memory usage: 2.7+ KB


In [112]:
def get_startdate(row):
    """
    日時と日付を連結して、日付型に変換
    """
    a = '0000' + str(row['測定投入時間'])
    mydate = row['測定投入日']
    mystr = str(mydate) + a[-4:]
    return pd.to_datetime(mystr,format='%Y%m%d%H%M')

def get_compdate(row):
    a = '0000' + str(row['測定完了時間'])
    mydate = row['測定完了日']
    mystr = str(mydate) + a[-4:]
    return pd.to_datetime(mystr,format='%Y%m%d%H%M')


df['startdate'] = df.apply(get_startdate,axis=1)
df['compdate'] = df.apply(get_compdate,axis=1)
df.head()

,オーダー№,測定投入日,測定投入時間,測定完了日,測定完了時間,測定機№,startdate,compdate
0,10FRE103S0,20200628,742,20200628,1108,LH26,2020-06-28 07:42:00,2020-06-28 11:08:00
1,10G1E20200,20200628,19,20200628,1834,LH25,2020-06-28 00:19:00,2020-06-28 18:34:00
2,10FVE20400,20200628,646,20200628,1855,LH12,2020-06-28 06:46:00,2020-06-28 18:55:00
3,10FSE104S0,20200628,1119,20200628,2058,LH26,2020-06-28 11:19:00,2020-06-28 20:58:00
4,10FFFA0300,20200628,1048,20200629,427,LH02,2020-06-28 10:48:00,2020-06-29 04:27:00


In [82]:
import datetime

In [7]:
#order_no = '10FRE103S0'
#start_date = datetime.datetime(2020,6,29,10,20)
#comp_date = datetime.datetime(2020,7,2,18,30)

In [113]:
def split_datetime(order_no,start_date,comp_date):
    """
    開始日時と終了日時を1日単位で分割する
    
    paramters:
        order_no : str orderno
        start_date: datetime 開始時間
        comp_date : datetime 終了時間
    returns
        lst : list 日毎に分割されたディクショナリ
        [{'オーダー№': '10FVE20100','from_date': datetime.datetime(2020, 6, 28, 21, 18),'to_date': datetime.datetime(2020, 6, 29, 0, 0)},
         {'オーダー№': '10FVE20100','from_date': datetime.datetime(2020, 6, 29, 0, 0),'to_date': datetime.datetime(2020, 6, 29, 12, 29)}]
    """
    range_date = comp_date - start_date
    #print(range_date)
    lst = []
    # 24時間以内でも日をまたぐことがあるので、＋２する
    for i in range(range_date.days + 2):
        from_date = start_date.replace(hour=0,minute=0) + datetime.timedelta(days=i)
        to_date = start_date.replace(hour=0,minute=0) + datetime.timedelta(days=i + 1)
        if start_date > from_date:
            from_date = start_date
        if comp_date < to_date:
            to_date = comp_date
        # 開始時間が終了時間より前のときにリストに追加。
        if from_date < to_date:
            lst.append({'オーダー№':order_no,'from_date':from_date,'to_date':to_date})
        #print(from_date,to_date,to_date-from_date)
    return lst

In [114]:
df.head()

,オーダー№,測定投入日,測定投入時間,測定完了日,測定完了時間,測定機№,startdate,compdate
0,10FRE103S0,20200628,742,20200628,1108,LH26,2020-06-28 07:42:00,2020-06-28 11:08:00
1,10G1E20200,20200628,19,20200628,1834,LH25,2020-06-28 00:19:00,2020-06-28 18:34:00
2,10FVE20400,20200628,646,20200628,1855,LH12,2020-06-28 06:46:00,2020-06-28 18:55:00
3,10FSE104S0,20200628,1119,20200628,2058,LH26,2020-06-28 11:19:00,2020-06-28 20:58:00
4,10FFFA0300,20200628,1048,20200629,427,LH02,2020-06-28 10:48:00,2020-06-29 04:27:00


In [115]:
order_no = '10FVE20100'
start_date = datetime.datetime(2020,6,28,21,18)
comp_date = datetime.datetime(2020,6,29,12,29)
split_datetime(order_no,start_date,comp_date)

[{'オーダー№': '10FVE20100',
  'from_date': datetime.datetime(2020, 6, 28, 21, 18),
  'to_date': datetime.datetime(2020, 6, 29, 0, 0)},
 {'オーダー№': '10FVE20100',
  'from_date': datetime.datetime(2020, 6, 29, 0, 0),
  'to_date': datetime.datetime(2020, 6, 29, 12, 29)}]

In [118]:
# 投入時間を日にちごとに分割したリストを作る
lst =[]
for index, row in df.iterrows():
    order_no, start_date, comp_date = row['オーダー№'],row['startdate'],row['compdate']
    mydic = split_datetime(order_no,start_date,comp_date)
    #print(order_no, start_date, comp_date)
    #print(mydic)
    lst = lst + split_datetime(order_no,start_date,comp_date)
print(lst[:5])

[{'オーダー№': '10FRE103S0', 'from_date': Timestamp('2020-06-28 07:42:00'), 'to_date': Timestamp('2020-06-28 11:08:00')}, {'オーダー№': '10G1E20200', 'from_date': Timestamp('2020-06-28 00:19:00'), 'to_date': Timestamp('2020-06-28 18:34:00')}, {'オーダー№': '10FVE20400', 'from_date': Timestamp('2020-06-28 06:46:00'), 'to_date': Timestamp('2020-06-28 18:55:00')}, {'オーダー№': '10FSE104S0', 'from_date': Timestamp('2020-06-28 11:19:00'), 'to_date': Timestamp('2020-06-28 20:58:00')}, {'オーダー№': '10FFFA0300', 'from_date': Timestamp('2020-06-28 10:48:00'), 'to_date': Timestamp('2020-06-29 00:00:00')}]


In [119]:
df_split = pd.DataFrame(lst)
df_split.head(10)

,from_date,to_date,オーダー№
0,2020-06-28 07:42:00,2020-06-28 11:08:00,10FRE103S0
1,2020-06-28 00:19:00,2020-06-28 18:34:00,10G1E20200
2,2020-06-28 06:46:00,2020-06-28 18:55:00,10FVE20400
3,2020-06-28 11:19:00,2020-06-28 20:58:00,10FSE104S0
4,2020-06-28 10:48:00,2020-06-29 00:00:00,10FFFA0300
5,2020-06-29 00:00:00,2020-06-29 04:27:00,10FFFA0300
6,2020-06-28 21:18:00,2020-06-29 00:00:00,10FVE20100
7,2020-06-29 00:00:00,2020-06-29 12:29:00,10FVE20100
8,2020-06-28 19:18:00,2020-06-29 00:00:00,10FWE20200
9,2020-06-29 00:00:00,2020-06-29 13:38:00,10FWE20200


In [121]:
# もとのデータフレームと日毎に分割したデータフレーをマージする
df2 = pd.merge(df,df_split,on='オーダー№')
df2.head(10)

,オーダー№,測定投入日,測定投入時間,測定完了日,測定完了時間,測定機№,startdate,compdate,from_date,to_date
0,10FRE103S0,20200628,742,20200628,1108,LH26,2020-06-28 07:42:00,2020-06-28 11:08:00,2020-06-28 07:42:00,2020-06-28 11:08:00
1,10G1E20200,20200628,19,20200628,1834,LH25,2020-06-28 00:19:00,2020-06-28 18:34:00,2020-06-28 00:19:00,2020-06-28 18:34:00
2,10FVE20400,20200628,646,20200628,1855,LH12,2020-06-28 06:46:00,2020-06-28 18:55:00,2020-06-28 06:46:00,2020-06-28 18:55:00
3,10FSE104S0,20200628,1119,20200628,2058,LH26,2020-06-28 11:19:00,2020-06-28 20:58:00,2020-06-28 11:19:00,2020-06-28 20:58:00
4,10FFFA0300,20200628,1048,20200629,427,LH02,2020-06-28 10:48:00,2020-06-29 04:27:00,2020-06-28 10:48:00,2020-06-29 00:00:00
5,10FFFA0300,20200628,1048,20200629,427,LH02,2020-06-28 10:48:00,2020-06-29 04:27:00,2020-06-29 00:00:00,2020-06-29 04:27:00
6,10FVE20100,20200628,2118,20200629,1229,LH26,2020-06-28 21:18:00,2020-06-29 12:29:00,2020-06-28 21:18:00,2020-06-29 00:00:00
7,10FVE20100,20200628,2118,20200629,1229,LH26,2020-06-28 21:18:00,2020-06-29 12:29:00,2020-06-29 00:00:00,2020-06-29 12:29:00
8,10FWE20200,20200628,1918,20200629,1338,LH25,2020-06-28 19:18:00,2020-06-29 13:38:00,2020-06-28 19:18:00,2020-06-29 00:00:00
9,10FWE20200,20200628,1918,20200629,1338,LH25,2020-06-28 19:18:00,2020-06-29 13:38:00,2020-06-29 00:00:00,2020-06-29 13:38:00


In [122]:
# 必要な項目だけ取り出す
df3 = df2.loc[:,['オーダー№','測定機№','startdate','compdate','from_date','to_date']]
df3.head()

,オーダー№,測定機№,startdate,compdate,from_date,to_date
0,10FRE103S0,LH26,2020-06-28 07:42:00,2020-06-28 11:08:00,2020-06-28 07:42:00,2020-06-28 11:08:00
1,10G1E20200,LH25,2020-06-28 00:19:00,2020-06-28 18:34:00,2020-06-28 00:19:00,2020-06-28 18:34:00
2,10FVE20400,LH12,2020-06-28 06:46:00,2020-06-28 18:55:00,2020-06-28 06:46:00,2020-06-28 18:55:00
3,10FSE104S0,LH26,2020-06-28 11:19:00,2020-06-28 20:58:00,2020-06-28 11:19:00,2020-06-28 20:58:00
4,10FFFA0300,LH02,2020-06-28 10:48:00,2020-06-29 04:27:00,2020-06-28 10:48:00,2020-06-29 00:00:00


In [132]:
# 利用時間を追加する
df3['use_time'] = df3['to_date']-df3['from_date']
df3.head()

,オーダー№,測定機№,startdate,compdate,from_date,to_date,use_time
0,10FRE103S0,LH26,2020-06-28 07:42:00,2020-06-28 11:08:00,2020-06-28 07:42:00,2020-06-28 11:08:00,03:26:00
1,10G1E20200,LH25,2020-06-28 00:19:00,2020-06-28 18:34:00,2020-06-28 00:19:00,2020-06-28 18:34:00,18:15:00
2,10FVE20400,LH12,2020-06-28 06:46:00,2020-06-28 18:55:00,2020-06-28 06:46:00,2020-06-28 18:55:00,12:09:00
3,10FSE104S0,LH26,2020-06-28 11:19:00,2020-06-28 20:58:00,2020-06-28 11:19:00,2020-06-28 20:58:00,09:39:00
4,10FFFA0300,LH02,2020-06-28 10:48:00,2020-06-29 04:27:00,2020-06-28 10:48:00,2020-06-29 00:00:00,13:12:00


In [130]:
# フィルターの条件を設定
cond = (df3['測定機№']=='LH26') & (datetime.datetime(2020,6,29,0,0) < df3['startdate']) & (df3['startdate']< datetime.datetime(2020,7,7,0,0))

In [133]:
# フィルタをかける
df3[cond]

,オーダー№,測定機№,startdate,compdate,from_date,to_date,use_time
22,10ECC30100,LH26,2020-06-29 13:20:00,2020-06-30 07:00:00,2020-06-29 13:20:00,2020-06-30 00:00:00,10:40:00
23,10ECC30100,LH26,2020-06-29 13:20:00,2020-06-30 07:00:00,2020-06-30 00:00:00,2020-06-30 07:00:00,07:00:00
37,10EBC30400,LH26,2020-06-30 07:30:00,2020-07-01 04:03:00,2020-06-30 07:30:00,2020-07-01 00:00:00,16:30:00
38,10EBC30400,LH26,2020-06-30 07:30:00,2020-07-01 04:03:00,2020-07-01 00:00:00,2020-07-01 04:03:00,04:03:00
69,10EEC30200,LH26,2020-07-03 02:13:00,2020-07-03 21:49:00,2020-07-03 02:13:00,2020-07-03 21:49:00,19:36:00
83,10EDC30100,LH26,2020-07-03 22:11:00,2020-07-04 13:48:00,2020-07-03 22:11:00,2020-07-04 00:00:00,01:49:00
84,10EDC30100,LH26,2020-07-03 22:11:00,2020-07-04 13:48:00,2020-07-04 00:00:00,2020-07-04 13:48:00,13:48:00
101,10EDC30200,LH26,2020-07-04 14:27:00,2020-07-05 10:08:00,2020-07-04 14:27:00,2020-07-05 00:00:00,09:33:00
102,10EDC30200,LH26,2020-07-04 14:27:00,2020-07-05 10:08:00,2020-07-05 00:00:00,2020-07-05 10:08:00,10:08:00


In [134]:
# queryを使って条件をつける
# https://note.nkmk.me/python-pandas-query/
df3.query('startdate < datetime.datetime(2020,6,29,0,0)')

,オーダー№,測定機№,startdate,compdate,from_date,to_date,use_time
0,10FRE103S0,LH26,2020-06-28 07:42:00,2020-06-28 11:08:00,2020-06-28 07:42:00,2020-06-28 11:08:00,0 days 03:26:00
1,10G1E20200,LH25,2020-06-28 00:19:00,2020-06-28 18:34:00,2020-06-28 00:19:00,2020-06-28 18:34:00,0 days 18:15:00
2,10FVE20400,LH12,2020-06-28 06:46:00,2020-06-28 18:55:00,2020-06-28 06:46:00,2020-06-28 18:55:00,0 days 12:09:00
3,10FSE104S0,LH26,2020-06-28 11:19:00,2020-06-28 20:58:00,2020-06-28 11:19:00,2020-06-28 20:58:00,0 days 09:39:00
4,10FFFA0300,LH02,2020-06-28 10:48:00,2020-06-29 04:27:00,2020-06-28 10:48:00,2020-06-29 00:00:00,0 days 13:12:00
5,10FFFA0300,LH02,2020-06-28 10:48:00,2020-06-29 04:27:00,2020-06-29 00:00:00,2020-06-29 04:27:00,0 days 04:27:00
6,10FVE20100,LH26,2020-06-28 21:18:00,2020-06-29 12:29:00,2020-06-28 21:18:00,2020-06-29 00:00:00,0 days 02:42:00
7,10FVE20100,LH26,2020-06-28 21:18:00,2020-06-29 12:29:00,2020-06-29 00:00:00,2020-06-29 12:29:00,0 days 12:29:00
8,10FWE20200,LH25,2020-06-28 19:18:00,2020-06-29 13:38:00,2020-06-28 19:18:00,2020-06-29 00:00:00,0 days 04:42:00
9,10FWE20200,LH25,2020-06-28 19:18:00,2020-06-29 13:38:00,2020-06-29 00:00:00,2020-06-29 13:38:00,0 days 13:38:00


In [43]:
df3.query('測定機№ == "LH26"')

SyntaxError: invalid character in identifier (<unknown>, line 1)

In [135]:
# 列名をアルファベットに
df4 = df3.rename(columns={'測定機№':'MachineNo','オーダー№':'OrderNo'})
df4.head()

,OrderNo,MachineNo,startdate,compdate,from_date,to_date,use_time
0,10FRE103S0,LH26,2020-06-28 07:42:00,2020-06-28 11:08:00,2020-06-28 07:42:00,2020-06-28 11:08:00,03:26:00
1,10G1E20200,LH25,2020-06-28 00:19:00,2020-06-28 18:34:00,2020-06-28 00:19:00,2020-06-28 18:34:00,18:15:00
2,10FVE20400,LH12,2020-06-28 06:46:00,2020-06-28 18:55:00,2020-06-28 06:46:00,2020-06-28 18:55:00,12:09:00
3,10FSE104S0,LH26,2020-06-28 11:19:00,2020-06-28 20:58:00,2020-06-28 11:19:00,2020-06-28 20:58:00,09:39:00
4,10FFFA0300,LH02,2020-06-28 10:48:00,2020-06-29 04:27:00,2020-06-28 10:48:00,2020-06-29 00:00:00,13:12:00


In [136]:
df4.query('MachineNo == "LH26"')

,OrderNo,MachineNo,startdate,compdate,from_date,to_date,use_time
0,10FRE103S0,LH26,2020-06-28 07:42:00,2020-06-28 11:08:00,2020-06-28 07:42:00,2020-06-28 11:08:00,03:26:00
3,10FSE104S0,LH26,2020-06-28 11:19:00,2020-06-28 20:58:00,2020-06-28 11:19:00,2020-06-28 20:58:00,09:39:00
6,10FVE20100,LH26,2020-06-28 21:18:00,2020-06-29 12:29:00,2020-06-28 21:18:00,2020-06-29 00:00:00,02:42:00
7,10FVE20100,LH26,2020-06-28 21:18:00,2020-06-29 12:29:00,2020-06-29 00:00:00,2020-06-29 12:29:00,12:29:00
22,10ECC30100,LH26,2020-06-29 13:20:00,2020-06-30 07:00:00,2020-06-29 13:20:00,2020-06-30 00:00:00,10:40:00
23,10ECC30100,LH26,2020-06-29 13:20:00,2020-06-30 07:00:00,2020-06-30 00:00:00,2020-06-30 07:00:00,07:00:00
37,10EBC30400,LH26,2020-06-30 07:30:00,2020-07-01 04:03:00,2020-06-30 07:30:00,2020-07-01 00:00:00,16:30:00
38,10EBC30400,LH26,2020-06-30 07:30:00,2020-07-01 04:03:00,2020-07-01 00:00:00,2020-07-01 04:03:00,04:03:00
69,10EEC30200,LH26,2020-07-03 02:13:00,2020-07-03 21:49:00,2020-07-03 02:13:00,2020-07-03 21:49:00,19:36:00
83,10EDC30100,LH26,2020-07-03 22:11:00,2020-07-04 13:48:00,2020-07-03 22:11:00,2020-07-04 00:00:00,01:49:00


In [137]:
# 複数条件のクエリ
df4.query('(MachineNo == "LH26") & (datetime.datetime(2020,6,29,0,0) < startdate < datetime.datetime(2020,7,7,0,0) )')

,OrderNo,MachineNo,startdate,compdate,from_date,to_date,use_time
22,10ECC30100,LH26,2020-06-29 13:20:00,2020-06-30 07:00:00,2020-06-29 13:20:00,2020-06-30 00:00:00,10:40:00
23,10ECC30100,LH26,2020-06-29 13:20:00,2020-06-30 07:00:00,2020-06-30 00:00:00,2020-06-30 07:00:00,07:00:00
37,10EBC30400,LH26,2020-06-30 07:30:00,2020-07-01 04:03:00,2020-06-30 07:30:00,2020-07-01 00:00:00,16:30:00
38,10EBC30400,LH26,2020-06-30 07:30:00,2020-07-01 04:03:00,2020-07-01 00:00:00,2020-07-01 04:03:00,04:03:00
69,10EEC30200,LH26,2020-07-03 02:13:00,2020-07-03 21:49:00,2020-07-03 02:13:00,2020-07-03 21:49:00,19:36:00
83,10EDC30100,LH26,2020-07-03 22:11:00,2020-07-04 13:48:00,2020-07-03 22:11:00,2020-07-04 00:00:00,01:49:00
84,10EDC30100,LH26,2020-07-03 22:11:00,2020-07-04 13:48:00,2020-07-04 00:00:00,2020-07-04 13:48:00,13:48:00
101,10EDC30200,LH26,2020-07-04 14:27:00,2020-07-05 10:08:00,2020-07-04 14:27:00,2020-07-05 00:00:00,09:33:00
102,10EDC30200,LH26,2020-07-04 14:27:00,2020-07-05 10:08:00,2020-07-05 00:00:00,2020-07-05 10:08:00,10:08:00


In [138]:
# sumを取る
df4.query('(MachineNo == "LH26") & (datetime.datetime(2020,6,29,0,0) < startdate < datetime.datetime(2020,7,7,0,0) )').sum()

OrderNo      10ECC3010010ECC3010010EBC3040010EBC3040010EEC3...
MachineNo                 LH26LH26LH26LH26LH26LH26LH26LH26LH26
use_time                                       3 days 21:07:00
dtype: object

In [139]:
# sumした結果を辞書にする
dict(df4.query('(MachineNo == "LH26") & (datetime.datetime(2020,6,29,0,0) < startdate < datetime.datetime(2020,7,7,0,0) )').sum())

{'OrderNo': '10ECC3010010ECC3010010EBC3040010EBC3040010EEC3020010EDC3010010EDC3010010EDC3020010EDC30200',
 'MachineNo': 'LH26LH26LH26LH26LH26LH26LH26LH26LH26',
 'use_time': Timedelta('3 days 21:07:00')}